In [ ]:
from pandas import read_csv
from datetime import datetime
from google.colab import files
import io
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf
from sklearn.metrics import classification_report

In [ ]:
# load data
 
uploaded = files.upload()

Saving PETR4_INPUT.csv to PETR4_INPUT.csv


In [ ]:
#Read CSV
Input = read_csv(io.BytesIO(uploaded['PETR4_INPUT.csv']), index_col = 'Date')
Train = Input[:int(0.7*len(Input))].copy()
Test = Input[int(0.7*len(Input)):].copy()

In [ ]:
# Put your inputs into a single list
input_cols = ['LOG_RETURN','VOLUME','MA', 'ADX', 'MACD', 'MFI']
Train['single_input_vector'] = Train[input_cols].apply(tuple, axis=1).apply(list)
Test['single_input_vector'] = Test[input_cols].apply(tuple, axis=1).apply(list)

In [ ]:
# Double-encapsulate list so that you can sum it in the next step and keep time steps as separate elements
Train['single_input_vector'] = Train.single_input_vector.apply(lambda x: [list(x)])
Test['single_input_vector'] = Test.single_input_vector.apply(lambda x: [list(x)])

In [ ]:
#Include Output
output_cols = ['OUTPUT']
Train['output_vector'] = Train[output_cols].apply(tuple, axis=1).apply(list)
Test['output_vector'] = Test[output_cols].apply(tuple, axis=1).apply(list)

In [ ]:
# Extract your training data
X_train_init = np.asarray(Train.single_input_vector)
# Use hstack to and reshape to make the inputs a 3d vector
X_train = np.hstack(X_train_init).reshape(len(Train),1,len(input_cols))
y_train = np.hstack(np.asarray(Train.OUTPUT)).reshape(len(Train),len(output_cols))


In [ ]:
# Extract your training data
X_test_init = np.asarray(Test.single_input_vector)
# Use hstack to and reshape to make the inputs a 3d vector
X_test = np.hstack(X_test_init).reshape(len(Test),1,len(input_cols))
y_test = np.hstack(np.asarray(Test.OUTPUT)).reshape(len(Test),len(output_cols))

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(259, 1, 6) (259, 1) (112, 1, 6) (112, 1)


In [ ]:
preds = []

for i in range(10):
  model = Sequential()
  model.add(LSTM(8,input_shape=(1, 6),return_sequences=True))
  model.add(LSTM(4,return_sequences=True))
  model.add(LSTM(2,return_sequences=False))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=750, batch_size=100,verbose=0,shuffle=True)
  y_pred = model.predict_classes(X_test)
  clas =classification_report(y_test, y_pred, output_dict=True)
  preds.append(clas)


In [ ]:
list_of_df = list()
for df in preds:
  list_of_df.append(pd.DataFrame(df).transpose())

In [ ]:
Final_df = sum(list_of_df)/10

In [ ]:
Final_df

,precision,recall,f1-score,support
0,0.794399,0.732727,0.749071,55.000000
1,0.768080,0.794737,0.770786,57.000000
accuracy,0.764286,0.764286,0.764286,0.764286
macro avg,0.781240,0.763732,0.759929,112.000000
weighted avg,0.781005,0.764286,0.760123,112.000000
